In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

from scipy import stats
import scipy.linalg as linalg

import math

In [21]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y

def gen_data(nobs, num_cov, a):
    xn = np.random.normal(scale=1., size=(nobs, num_cov))
    e = np.random.normal(loc=0.0, scale=1.0, size=(nobs))
    yn = (xn.sum(axis=1) * a/num_cov) + e
    return yn, xn


def setup_model(model1_fit,yn,xn):
    """setup models for ease"""
    param1 = (model1_fit.params)
    model1_deriv = OLS_loglike(yn,sm.add_constant(xn))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    return ll1,grad1,hess1,param1


def setup_test(yn,xn,a=.1):
    lls = []
    grads = []
    hesss = []
    params = []
    ols = sm.OLS(yn, sm.add_constant(xn)).fit()
    ridge = sm.OLS(yn, sm.add_constant(xn)).fit_regularized(method='elastic_net', alpha=a, L1_wt=0.0)
    for model in [ols,ridge]:
        ll,grad,hess,param = setup_model(model,yn,xn)
        lls.append(ll)
        params.append(param)
        grads.append(grad)
        hesss.append(hess)   

    return lls[0],grads[0],hesss[0],params[0],lls[1],grads[1],hesss[1],params[1]


def compute_eigen2(ll1,grad1,hess1,params1,ll2,grad2,hess2,params2):
    
    n = ll1.shape[0]
    hess1 = hess1/n
    hess2 = hess2/n

    k1 = params1.shape[0]
    k2 = params2.shape[0]
    k = k1 + k2
    
    #A_hat:
    A_hat1 = np.concatenate([hess1,np.zeros((k2,k1))])
    A_hat2 = np.concatenate([np.zeros((k1,k2)),-1*hess2])
    A_hat = np.concatenate([A_hat1,A_hat2],axis=1)

    #B_hat, covariance of the score...
    B_hat =  np.concatenate([grad1,-grad2],axis=1) #might be a mistake here..
    B_hat = np.cov(B_hat.transpose())

    #compute eigenvalues for weighted chisq
    sqrt_B_hat= linalg.sqrtm(B_hat)
    W_hat = np.matmul(sqrt_B_hat,linalg.inv(A_hat))
    W_hat = np.matmul(W_hat,sqrt_B_hat)
    V,W = np.linalg.eig(W_hat)

    return V

def compute_stage1(ll1,grad1,hess1,params1,ll2, grad2,hess2,params2):
    nsims = 5000
    
    k1 = params1.shape[0]
    k2 = params2.shape[0]
    k = k1 + k2
    
    V = compute_eigen2(ll1,grad1,hess1,params1,ll2, grad2,hess2,params2)
    np.random.seed()
    Z0 = np.random.normal( size=(nsims,k) )**2
    
    return np.matmul(Z0,V*V)

def two_step_test(ll1,grad1,hess1,params1,ll2,grad2,hess2,params2,biascorrect=True):
    stage1_distr = compute_stage1(ll1,grad1,hess1,params1,ll2, grad2,hess2,params2)
    print('denom bias:', stage1_distr.mean())
    nobs = ll1.shape[0]
    omega0 = np.sqrt( (ll1 -ll2).var())
    omega = np.sqrt( (ll1 -ll2).var() + stage1_distr.mean())#set up stuff
    V =  compute_eigen2(ll1,grad1,hess1,params1,ll2,grad2,hess2,params2)
    llr = (ll1 - ll2).sum()
    if biascorrect:
        print('num bias:',V.sum()/2)
        llr = llr + V.sum()/(2) #fix the test...
    test_stat = llr/(omega*np.sqrt(nobs))
    print('llr:',llr.sum(),'omega0:',omega0, 'fixed', omega, 'test_stat:',test_stat)
    
    stage1_res = ( nobs*omega**2 >= np.percentile(stage1_distr, 95, axis=0) )
    return (1*(test_stat >= 1.96) + 2*( test_stat <= -1.96))*stage1_res

y,X = gen_data(1000, 10, 1)
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_test(y,X)
two_step_test(ll1,grad1,hess1,params1,ll2,grad2,hess2,params2)

denom bias: 0.021506523529193394
num bias: 0.007307873257857934
llr: 0.38733691667435627 omega0: 0.02701296357125336 fixed 0.14911815359001485 test_stat: 0.08214069508434663


0

In [ ]:
y,X = gen_data(1000, 10, 5)
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_test(y,X)
two_step_test(ll1,grad1,hess1,params1,ll2,grad2,hess2,params2)
y,X = gen_data(1000, 5, 1)
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_test(y,X)
two_step_test(ll1,grad1,hess1,params1,ll2,grad2,hess2,params2)
y,X = gen_data(1000, 5, 5)
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_test(y,X)
two_step_test(ll1,grad1,hess1,params1,ll2,grad2,hess2,params2)

In [30]:
y,X = gen_data(1000, 25, 1)
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_test(y,X,a=.2)
two_step_test(ll1,grad1,hess1,params1,ll2,grad2,hess2,params2)
y,X = gen_data(1000, 25, 1)
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_test(y,X,a=.2)
two_step_test(ll1,grad1,hess1,params1,ll2,grad2,hess2,params2)
y,X = gen_data(1000, 10, 5)
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_test(y,X,a=.2)
two_step_test(ll1,grad1,hess1,params1,ll2,grad2,hess2,params2)

denom bias: 0.14775883006698068
num bias: 0.010855989484741615
llr: 1.2616432089246492 omega0: 0.04801906006783997 fixed 0.3873818015818238 test_stat: 0.1029905410732842
denom bias: 0.10458536820969887
num bias: 0.012739838250595343
llr: 0.8821830669704532 omega0: 0.040167906068793516 fixed 0.32588161790387976 test_stat: 0.08560494521916624
denom bias: 1.5582900530930717
num bias: -0.05965842081608089
llr: 28.081708547971505 omega0: 0.23912839070104155 fixed 1.2710123682845662 test_stat: 0.6986726629612768


0

In [44]:
#try it with seperate x?
y,X = gen_data(1000, 10, 5)
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_test(y,X,a=.2)
y,X = gen_data(1000, 10, 5)
_,_,_,_,ll2,grad2,hess2,params2 = setup_test(y,X,a=.2)
two_step_test(ll1,grad1,hess1,params1,ll2,grad2,hess2,params2)
#still a numerator bias?

denom bias: 21.686856067920786
num bias: -0.10751639659489631
llr: 41.946764129479156 omega0: 1.0247192437713488 fixed 4.768323143042648 test_stat: 0.27818440811115525


0